# Analysizing Crime Data

In [1]:
# Dependencies and Setup
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from statsmodels.formula.api import glm
import statsmodels.api as sm
import scipy.stats as stats

In [2]:
file = "SummaryData_Pop.csv"
crime = pd.read_csv(file)
crime.head()

,City,Year,Month,DayofWeek,StartTime,MapCrime,MapWeather,Population,Count
0,atlanta,2014,1,Friday,0,Auto-Theft,Clear,5455053,1
1,atlanta,2014,1,Friday,0,Theft,Clear,5455053,9
2,atlanta,2014,1,Friday,0,Theft,Overcast,5455053,1
3,atlanta,2014,1,Friday,0,Theft,Partly Cloudy,5455053,2
4,atlanta,2014,1,Friday,0,Violence,Overcast,5455053,1


In [5]:
fields = ["City", "MapWeather", "Count"]
crime1 = crime[fields].copy()
fields = ["City", "MapWeather"]
crime_sum = crime1.groupby(fields).agg("sum")
crime_sum.head()

Count
City    MapWeather          
atlanta Clear          42811
        Mostly Cloudy  27364
        Overcast       19430
        Partly Cloudy  36270
        Rain           15642

In [7]:
crime_sum.to_csv("crimebyweather.csv", sep=',', encoding='utf-8')

In [17]:
file = "weather_df.csv"
weather = pd.read_csv(file)
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218695 entries, 0 to 218694
Data columns (total 5 columns):
city          218695 non-null object
weather       218695 non-null object
date          218695 non-null object
time          218695 non-null int64
MapWeather    218695 non-null object
dtypes: int64(1), object(4)
memory usage: 8.3+ MB


In [18]:
fields = ["city", "MapWeather", "time"]
wthr1 = weather[fields].copy()
fields = ["city", "MapWeather"]
wthr2 = wthr1.groupby(fields).agg("count")
wthr2 = wthr2.rename(columns = {"time" : "Count"})
wthr2.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 29 entries, (atlanta, Clear) to (los_angeles, Rain)
Data columns (total 1 columns):
Count    29 non-null int64
dtypes: int64(1)
memory usage: 522.0+ bytes


In [19]:
wthr2.to_csv("weatherbycity.csv", sep=',', encoding='utf-8')

In [ ]:
df3 = pd.merge(df, df2, how='inner', left_on = ["City","MapWeather","StartTime"], right_on = ["city","MapWeather","time"])
df3.head()

In [ ]:
df3["ratio"] = df3["Count_x"] / df3["Count_y"]
df_glm = df3.copy()
df_glm.head()

In [ ]:
obj_list = df3.select_dtypes('object').apply(pd.Series.nunique, axis = 0)
le = LabelEncoder()

for i in obj_list.index:
    try:
        le.fit(df_glm[i])
        df_glm[i] = le.transform(df_glm[i])
    except:
        df_glm[i] = pd.get_dummies(df_glm[i])

In [ ]:
i = 6
d1 = df_glm.groupby(df_glm.columns[i])
plt.plot(d1["ratio"].mean())
plt.title(df_glm.columns[i])
plt.show()

In [ ]:
a = df.columns[0]
b = df.columns[1]
c = df.columns[2]
d = df.columns[3]
e = df.columns[4]
f = df.columns[5]
g = df.columns[6]

form1 = "ratio ~ " + a + " + " +  b + " + " + c + " + " + d + " + " + e + " + " + g
print(form1)
form2 = "ratio ~ " + a + " + " +  b + " + " + c + " + " + d + " + " + e
print(form2)

In [ ]:
#model_ols = glm(formula = form1, data = df_glm, family = sm.families.Gaussian()).fit()
#print(model_ols.summary())